In [3]:
BLS12_381_FR = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001



In [4]:
P = BLS12_381_FR
A = 0xec16b4441608ea5c955282418f882a4bdfd42168ccf447394f1d4546b9ca4c5
B = 0x547e4510e6c9d8b086d8a258704296b37ebaa877ee6fe9e43b4a3a84f9854f5f

print(hex((A*B)%P))

0x895e85c7af747dfe9b49261cfd69000928e038bd867259b7f7ae1ff84681614


In [5]:
P = BLS12_381_FR
A = 0x45ac411bed701ce7f02188e01653c6ccd89bf96021e9d95b52b2a2ba23d89d20
B = 0x3683eb4ea07253bd6c974375cec2fbe36b8a69643d13a7db647ae6a66f793473

print(hex((A*B)%P))

0xd5039d89dbd10a47ee3219ae9f0bb6c1996fe5967bfaf40f89e87ed57986b20


In [8]:
import numpy as np
from gmpy2 import mpz

# Function to read bigint<4> values from a binary file
def read_bigint_file(filename, element_size=32):
    values = []
    with open(filename, "rb") as f:
        while True:
            data = f.read(element_size)
            if not data:
                break
            value = int.from_bytes(data, byteorder='little', signed=False)
            values.append(mpz(value))
    return values

# Function to perform NTT using Negative Wrapped Convolution (NWC)
def ntt_nwc(values, omega, modulus):
    n = len(values)
    result = [mpz(0)] * n
    for k in range(n):
        for j in range(n):
            result[k] += values[j] * pow(omega, (j * k) % n, modulus)
            result[k] %= modulus
    return result

# Function to perform polynomial multiplication using NWC
def polynomial_multiplication_ntt(a, b, omega, modulus):
    n = len(a)
    # Forward NTT on both polynomials
    ntt_a = ntt_nwc(a, omega, modulus)
    ntt_b = ntt_nwc(b, omega, modulus)
    
    # Pointwise multiplication in NTT domain
    ntt_c = [(ntt_a[i] * ntt_b[i]) % modulus for i in range(n)]
    
    # Inverse NTT (using omega^{-1})
    omega_inv = pow(omega, -1, modulus)
    c = ntt_nwc(ntt_c, omega_inv, modulus)
    
    # Normalize the result by dividing by n (modular inverse of n)
    n_inv = pow(n, -1, modulus)
    c = [(c[i] * n_inv) % modulus for i in range(n)]
    
    return c

# Main function
def main():
    # Read input polynomials from binary files
    a = read_bigint_file("../build/data/input_a.txt")
    b = read_bigint_file("../build/data/input_b.txt")

    # Parameters for NTT
    modulus = mpz('0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001')
    omega = pow(3, (modulus - 1) // len(a), modulus)  # Primitive 2n-th root of unity

    # Perform polynomial multiplication using NWC and NTT
    result = polynomial_multiplication_ntt(a, b, omega, modulus)

    # Print the result
    for i, coeff in enumerate(result):
        print(f"Coefficient {i}: {hex(coeff)}")

    

if __name__ == "__main__":
    main()


Coefficient 0: 0x6f0e70ae22fac4a3f1dddf3930c4dea271bff47db7633b078e8fe329a20a8409
Coefficient 1: 0x6f0e70ae22fac4a3f1dddf3930c4dea271bff47db7633b078e8fe329a20a8409
